# Partial Delivery I – Genetic Programming (GP)

In this notebook, we address the problem of predicting **crude protein** weight using the other variables in the dataset `sustavianfeed.xlsx`. This is achieved through **Genetic Programming (GP)**, using symbolic regression to evolve a mathematical expression that approximates the target variable.

The notebook is organized into the following sections:

- [1. Imports and Configurations](#1-imports-and-configurations)  
- [2. Data Wrangling](#2-data-wrangling)  
- [3. Initialization](#3-initialization)  
- [4. Hyperparameter Tuning](#4-hyperparameter-tuning)
- [5. Model Assessment](#5-model-assessment)
- [6. Discussion](#6-discussion)


## 1. Imports and Configurations
- Imports the necessary libraries.
- Validates and loads configuration `.json` file.

### Imports

Below the list of imports used for this notebook.

In [1]:
# Standard Library
import os
from collections import defaultdict

# Data-handling
import numpy as np
import pandas as pd

# Plotting
import plotly.graph_objects as go

# Validation
from sklearn.model_selection import KFold

# Tensors
import torch

# Local
from nel_utils import json_parser, plotter, imputer, caller, selector

### Configurations

When possible, our pattern of choice is to load our configurations from a `.json` file. This ensures easy reproducibility across all notebooks that make use of configurations, as well as making versioning simpler.

Our notebook requires **four types of input arguments**:

- **System and visualization parameters:** Control the aesthetics of plots.  
- **Solution Space parameters:** Define the problem being solved.  
- **Solver parameters:** Specify optimization and algorithmic settings.  
- **Model hyperparameters:** Configure model-specific options.
- **Logger parameters:** Manages model and solver logging ouputs.  

In [2]:
# Configuration File
config_path = '../configs/gp_config.json'
config = json_parser.load_config(
    config_path,
    algorithm_schema='gp'
)

In [3]:
# Assigning them out to their respective key entries
sys_params = config.get('system_params')
data_params = config.get('data_params')
solver_params = config.get('solver_params')
grid_params = config.get('grid_params')
log_params = config.get('logging_params')

In [4]:
# Ensuring logging

if not os.path.exists(log_params['log_path']):
    os.makedirs(log_params['log_path'])

## 2. Data Wrangling

We now ingest, and wrangle the data.

In [5]:
csv_file = '../data/sustavianfeed.xlsx' 
df = pd.read_excel(csv_file, index_col='WING TAG', na_values="/")
df.head()

,WEIGHT,HOT CARCASS WEIGHT,CARCASS WEIGHT WITH HEAD AND LEGS,COLD CARCASS WEIGHT,BREAST WEIGHT (2),THIGH WEIGHT (2),SPLEEN,LIVER,HEART,INTESTINE,EMPTY MUSCULAR STOMACH,GLANDULAR STOMACH,CRUDE PROTEIN,ETHER EXTRACT
WING TAG,,,,,,,,,,,,,,
G403,2223.3,1429.6,1725.6,1394.0,214.0,489.4,3.716,38.636,9.305,123.171,NaN,13.170,86.105469,0.38
G439,2201.9,1450.2,1769.8,1405.4,236.0,538.7,3.494,34.725,10.084,71.800,45.273,9.781,86.143472,1.66
G454,2159.9,1398.4,1724.9,1461.7,241.8,512.1,4.023,31.932,10.635,61.380,NaN,6.217,86.416898,0.98
G465,2198.7,1473.9,1800.4,1425.1,227.7,549.9,3.087,32.326,11.927,64.879,35.861,8.358,85.959935,1.10
G428,2003.2,1291.2,1581.6,1260.1,224.7,473.2,3.723,30.105,9.855,68.562,36.526,7.572,81.693637,6.34


And inspect the resulting distributions.

In [6]:
plotter.plot_features(
    df, 
    df.columns,
    plot_type= 'Histogram',
    num_columns=3, 
    plot_title='Feature Distributions'
)

In [7]:
plotter.plot_features(
    df, 
    df.columns, 
    plot_type= 'Box',
    num_columns=3, 
    plot_title='Feature Distributions'
)

#### **On outliers**

Inspection of the distributions above informs us that the variables appear more or less normal, with very few abnormal values. Despite the possibility of treating these values, this is fundamentally a problem of learning the best representation of the target variable (**WEIGHT**) using other features. There is no strong reason to assume that outliers, even if present, necessarily break the proportional relationships inherent to animal morphology.

Moreover, the **WEIGHT** variable follows a visibly normal distribution. Barring pathological cases—such as an animal with near-zero fitness, or, say, a chicken that has suffered severe physical trauma and lost a significant portion of its body mass—there is no biological or statistical rationale to exclude or transform these points.

Thus, we retain all observations, positing — for now — that outliers are not noise to be removed but potential expressions of the natural variation in the population, which a good model would learn to account for. Albeit, naturally, we will revise this approach in a later instance.


#### **On missing values**

Given the well behaved distributions of our features, we can easily and confidently use a Nearest Neighbours approach to fill our missing values.

In [8]:
df = imputer.knn_impute(df, n_neighbors=5)

Inspecting the resulting imputations, they map more or less to the median value, so an even simpler strategy could have been pursued with **SimpleImputer**, and **mode=median**.

## 3. Initialization

- Creates training splits
- Initializes the Genetic Programming (GP) model.
- Sets up logging mechanisms for tracking progress and results.
- Prepares validation logic and supporting utilities.

### Folds and splits

In [9]:
# Get the seed and splits
seed = sys_params['random_seed']
k_outter = data_params['inner_folds']
k_inner = data_params['inner_folds']

# Make the folds
outter_cv = KFold(
    n_splits=k_outter, 
    random_state=seed, 
    shuffle=True
)

inner_cv = KFold(
    n_splits=k_inner, 
    random_state=seed, 
    shuffle=True
)

### Setting the target

We now set out target and convert to Tensors.

In [10]:
X = torch.tensor(
    np.delete(
        df.values, 
        -2, 
        axis=1
    ), 
    dtype=torch.float32
)  

y = torch.tensor(
    df.values[:, -2], 
    dtype=torch.float32
)  

X, y

(tensor([[2.2233e+03, 1.4296e+03, 1.7256e+03,  ..., 4.7176e+01, 1.3170e+01,
          3.8000e-01],
         [2.2019e+03, 1.4502e+03, 1.7698e+03,  ..., 4.5273e+01, 9.7810e+00,
          1.6600e+00],
         [2.1599e+03, 1.3984e+03, 1.7249e+03,  ..., 4.9286e+01, 6.2170e+00,
          9.8000e-01],
         ...,
         [2.6482e+03, 1.7229e+03, 2.0500e+03,  ..., 4.9700e+01, 7.3320e+00,
          1.9100e+00],
         [2.2626e+03, 1.4985e+03, 1.8139e+03,  ..., 4.6816e+01, 5.4250e+00,
          2.2100e+00],
         [2.7043e+03, 1.6988e+03, 2.0388e+03,  ..., 7.5767e+01, 7.4200e+00,
          1.5100e+00]]),
 tensor([86.1055, 86.1435, 86.4169, 85.9599, 81.6936, 87.5331, 86.8410, 88.2468,
         87.5399, 88.1924, 87.7862, 86.2464, 86.1611, 86.5115, 86.8366, 84.9951,
         86.3812, 88.5102, 85.0129, 85.9784, 88.9662, 84.7507, 87.1134, 87.2657,
         85.5097, 87.1627, 84.8371, 86.8986, 87.4809, 88.2095, 83.4647, 86.3373,
         86.5872, 87.8968, 88.5642, 87.4188, 87.9764, 84.6742, 73.

In [11]:
total_instances = X.shape[0]
outer_test_size = total_instances // k_outter
outer_train_size = total_instances - outer_test_size
inner_val_size = outer_train_size // k_inner
inner_train_size = outer_train_size - inner_val_size

print(f'Total Instances:\t{total_instances}\n--')
print(f'Outer Train set:\t{outer_train_size}')
print(f'Test set:\t\t{outer_test_size}\n--')
print(f'Inner Train set:\t{inner_train_size}')
print(f'Validation set:\t\t{inner_val_size}\n')


Total Instances:	96
--
Outer Train set:	77
Test set:		19
--
Inner Train set:	62
Validation set:		15



## 4. Model fitting with Hyperparameter Tuning

- Runs the symbolic regression, making use of nested cross validation with kfolds to optimize for model parameters and hyperparameter.
- Evolves expressions to minimize prediction error.
- Tracks progress and logs intermediate results.

### Splits

In [12]:
# Enumerating the outer splits
data_cv_outer = [
    [learning_ix, test_ix] for learning_ix, test_ix in outter_cv.split(X, y)
]

In [13]:
# Asserting Outer CV
learning_ix, test_ix = data_cv_outer[0]

X_learning = X[learning_ix]
y_learning = y[learning_ix]
X_test = X[test_ix]
y_test = y[test_ix]

print('\n' + '-' * 41 + '\n')
print(f'Outer CV\nLearning shape: {X_learning.shape}\nTest shape: {X_test.shape}\n')


-----------------------------------------

Outer CV
Learning shape: torch.Size([76, 13])
Test shape: torch.Size([20, 13])



In [14]:
# Enumerating the inner splits
data_cv_inner = [[train_ix, val_ix] for train_ix, val_ix in inner_cv.split(X_learning, y_learning)]

### Model Fitting

In [ ]:
# Running the model
results = []
for i_inner, (train_ix, val_ix) in enumerate(data_cv_inner):
    print('-----\nInner CV {}'.format(i_inner))
    
    # Data
    X_train, y_train = X_learning[train_ix], y_learning[train_ix]
    X_val, y_val = X_learning[val_ix], y_learning[val_ix]

    print(f'Training shape: {X_train.shape}\nValidation shape: {X_val.shape}\n')
    
    solver_params.update({
        'X_train': X_train, 'y_train': y_train,
        'X_test': X_val, 'y_test': y_val
    })

    # Fit
    log_path = f"{log_params['log_path']}_{solver_params['algorithm']}_{str(i_inner)}.csv"

    if os.path.exists(log_path):
        os.remove(log_path)

    solver_params.update({'log_path': log_path})
    
    res = caller.call_model(
        solver_params, 
        grid_params,
        seed=(sys_params['random_seed'] + i_inner)    
    )
    
    # Log
    results.append(res)

-----
Inner CV 0
Training shape: torch.Size([60, 13])
Validation shape: torch.Size([16, 13])

Verbose Reporter
-----------------------------------------------------------------------------------------------------------------------------------------
|         Dataset         |  Generation  |     Train Fitness     |       Test Fitness       |        Timing          |      Nodes       |
-----------------------------------------------------------------------------------------------------------------------------------------
|     sustavianfeed       |       0      |   52.40753936767578   |   50.06377410888672      |   0.0037801265716552734|      3           |
|     sustavianfeed       |       1      |   50.53290939331055   |   48.19805908203125      |   0.002821207046508789 |      3           |
|     sustavianfeed       |       2      |   50.53290939331055   |   48.19805908203125      |   0.001669168472290039 |      3           |
|     sustavianfeed       |       3      |   50.5329093933105

## 5. Model Assessment

- Visualize train, validation results for convergence and overfit.
- Assesses symbolic expressions for bloat.
- Assess best model based on above criteria

In [ ]:
# Plotting Settings
dataset_name = solver_params['dataset_name']
log_dir = log_params['log_path']
train_color = sys_params['train_color']
test_color = sys_params['test_color']

In [ ]:
# Plot settings
df_log = []
for i_inner in range(k_inner):
    tmp = pd.read_csv(f"{log_dir}_{solver_params['algorithm']}_{i_inner}.csv", header=None)
    tmp['cv'] = i_inner
    df_log.append(tmp)
df_log = pd.concat(df_log, ignore_index=True)

### Train, Validation Average Fold Results

In [ ]:
plotter.make_evolution_plot(
    df_log,  plot_title = f"GP - Train vs Validation Fitness ('{dataset_name}' dataset)", var='rmse')

### By Fold, Test Fitness

In [ ]:
rmse_by_config = defaultdict(list)

for split in results:
    rmse_train = []
    rmse_test = []
    
    for result in split:
        key = ''
        for k, v in result['dynamic_params'].items():
            key += k+': '+str(v)+' <br /> '
        rmse_by_config[key].append(result['rmse_test'])

fig = go.Figure()
for config, rmse_values in rmse_by_config.items():
    fig.add_trace(go.Box(
        y=rmse_values,
        boxpoints='all',
        jitter=0.5,
        pointpos=0,
        line=dict(color='orange'),
        name=config
    ))

fig.update_layout(
    title= solver_params['dataset_name'] +' dataset',
    xaxis_title='',
    yaxis_title='Test RMSE',
    height=500, width=25000,
    xaxis_tickangle=-45,
    yaxis_range=[0,None],
    margin=dict(l=50, r=50, t=50, b=20),
    showlegend=False,
    template='plotly_white'
)

fig.show()

### Program Size

In [ ]:
plotter.make_evolution_plot(
    df_log,  
    plot_title = f"GP - Train vs Test Fitness ('{dataset_name}' dataset)", 
    var='size'
)

### Discussion

Our GP model demonstrates clear signs of premature convergence and consistently sub-optimal fitness values. The population exhibits low diversity and converges rapidly. This suggests that increasing tree depth or incorporating alternative variation operators might have promoted greater exploration and delayed convergence,

The model also shows chronic **overfitting**. Although the absolute error gap between training and validation is small, generalization fails across almost all most runs.

Lastly, **bloat**—while present—follows a roughly linear trend and remains relatively controlled. Nonetheless, stronger parsimony pressure could further improve model interpretability.